In [1]:
!cat container/Dockerfile

In [2]:
%%sh

# The name of our algorithm
algorithm_name=sagemaker-decision-trees

cd container

chmod +x decision_trees/train
chmod +x decision_trees/serve

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-west-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
aws ecr get-login-password --region ${region}|docker login --username AWS --password-stdin ${fullname}

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

chmod: cannot access ‘decision_trees/train’: No such file or directory
chmod: cannot access ‘decision_trees/serve’: No such file or directory
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



Login Succeeded


#0 building with "default" instance using docker driver

#1 [internal] load build definition from Dockerfile
#1 transferring dockerfile: 31B done
#1 DONE 0.0s
ERROR: failed to solve: the Dockerfile cannot be empty
Error response from daemon: No such image: sagemaker-decision-trees:latest


The push refers to repository [034362035459.dkr.ecr.us-east-2.amazonaws.com/sagemaker-decision-trees]


An image does not exist locally with the tag: 034362035459.dkr.ecr.us-east-2.amazonaws.com/sagemaker-decision-trees


CalledProcessError: Command 'b'\n# The name of our algorithm\nalgorithm_name=sagemaker-decision-trees\n\ncd container\n\nchmod +x decision_trees/train\nchmod +x decision_trees/serve\n\naccount=$(aws sts get-caller-identity --query Account --output text)\n\n# Get the region defined in the current configuration (default to us-west-2 if none defined)\nregion=$(aws configure get region)\nregion=${region:-us-west-2}\n\nfullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"\n\n# If the repository doesn\'t exist in ECR, create it.\naws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1\n\nif [ $? -ne 0 ]\nthen\n    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null\nfi\n\n# Get the login command from ECR and execute it directly\naws ecr get-login-password --region ${region}|docker login --username AWS --password-stdin ${fullname}\n\n# Build the docker image locally with the image name and then push it to ECR\n# wi

In [8]:
# S3 prefix
prefix = "winequality-red"

# Define IAM role
import boto3
import re

import os
import numpy as np
import pandas as pd
from sagemaker import get_execution_role

role = get_execution_role()

In [4]:
import sagemaker as sage
from time import gmtime, strftime

sess = sage.Session()

In [9]:
WORK_DIRECTORY = "data"

data_location = sess.upload_data(WORK_DIRECTORY, key_prefix=prefix)

In [12]:
from sagemaker.sklearn import SKLearn

sk_estimator = SKLearn(
    entry_point="container/decision_trees/predictor.py",
    role=role,
    instance_count=1,
    instance_type="ml.c5.xlarge",
    py_version="py3",
    framework_version="1.0-1",
    hyperparameters={"estimators": 20},
)

In [13]:
sk_estimator.fit(data_location)

INFO:sagemaker:Creating training-job with name: sagemaker-scikit-learn-2024-08-28-05-12-59-384


2024-08-28 05:13:01 Starting - Starting the training job...
2024-08-28 05:13:35 Downloading - Downloading input data...
2024-08-28 05:13:50 Downloading - Downloading the training image.....2024-08-28 05:14:39,309 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2024-08-28 05:14:39,313 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-08-28 05:14:39,316 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2024-08-28 05:14:39,333 sagemaker_sklearn_container.training INFO     Invoking user training script.
2024-08-28 05:14:39,571 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-08-28 05:14:39,574 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2024-08-28 05:14:39,593 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-08-28 05:14:39,596 sagemaker-training-toolkit IN

In [16]:
predictor = sk_estimator.deploy(1, "ml.m4.xlarge")

INFO:sagemaker:Creating model with name: sagemaker-scikit-learn-2024-08-28-05-16-22-972


ClientError: An error occurred (ValidationException) when calling the CreateModel operation: Could not find model data at s3://sagemaker-us-east-2-034362035459/sagemaker-scikit-learn-2024-08-28-05-12-59-384/output/model.tar.gz.